### Принцип работы

Для задачи классификации важно, чтобы классы были примерно одного размера. При наличии нескольких корпусов, каждый из которых содержит разное количество токенов и предложений, решить задачу балансировки размеров действительно необходимо. 

Балансировать корпуса нужно с ориентацией на то, какими будут сэмплы. То есть, например, неправильно балансировать по количеству токенов, если каждый сэмпл это отдельное предложение, потому что в разных корпусах длины предложений могут отличаться.

Поскольку предыдущие задачи работали только с предложениями, то балансировщик будет работать на уровне предложений. Пусть есть последовательность корпусов $c_1, c_2 \ldots c_n$, через $|c_i|$ обозначим количество предложений в корпусе. На начальном этапе очень вероятно, что $|c_1| \neq |c_2| \ldots \neq |c_n|$, хочется $|c_1| \approx |c_2| \ldots \approx |c_n|$ с хорошей точностью. Давайте для каждого корпуса проитерируемся по его предложениям и у каждого вычислим целый логарифм длины, тогда все предложения разобьются на бакеты 

$$\large B^k_{L} = \{ s \space | \space s \in c_k \space \wedge \space \lfloor \log (|s|) \rceil = L\}$$

где $k \in [1 \ldots n]$, а разброс значений $L$ зависит от основания логарифма. Затем все бакеты собираются в один датафрейм. Понижая основание логарифма можно увеличивать количество бакетов, при единичном основании, например, все предложения просто разобьются по длине. Это может быть полезно, если при дльнейшей обработке захочется в большей степени контролировать длину предложений.

<img src="images/balancer.png" width="800"/>

После разбиения на бакеты в дело вступает балансировщик. Он принимает на вход датафрейм с бакетами и число $Q$ - верхняя граница на размер бакета. И все что он делает, это из каждого бакета рандомно выбирает $Q$ предложений, а при построении индекса фильтрует предложения, которые не попадают в выборку. Это всегда будет работать корректно, поскольку каждое предложение принадлежит какому-то корпусу, и у него можно посчитать целый логарифм длины, а корпус и логарифм вместе определяют адрес бакета.

### Пример использования

Код балансировщика находится в ```tg/grammar_ru/ml/corpus/corpus_balancer.py```

In [1]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.ml.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd

На первом шаге формируем список корпусов, из которых будет собираться бандл и которые нужно сбалансировать

In [2]:
CORPUS_NAMES = [
    "books.base.zip", 
    "ficbook.base.zip"
]

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [3]:
CORPUS_LIST

[PosixPath('/home/dabdya/grammar_ru/data-cache/corpus/books.base.zip'),
 PosixPath('/home/dabdya/grammar_ru/data-cache/corpus/ficbook.base.zip')]

Проверим исходную сбалансированность, имеет ли смысл вообще делать что-то дальше

In [6]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [7]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

100%|███████████████████████████████████████| 2533/2533 [00:43<00:00, 58.50it/s]


books.base.zip = 903668 sentences


100%|█████████████████████████████████████████| 998/998 [00:14<00:00, 67.08it/s]

ficbook.base.zip = 120270 sentences


Видно что есть большая несбалансированность, поэтому фиксируем праметры для формирования бакетов и начинаем билдинг

In [8]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/example.parquet"

In [9]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|███████████████████████████████████████| 3531/3531 [04:08<00:00, 14.21it/s]


In [10]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65
ficbook.base.zip/0,"[73552, 93817, 117139, 117186, 117216, 117370,...",449
ficbook.base.zip/1,"[0, 10144, 10145, 10170, 10176, 10178, 10180, ...",13029
ficbook.base.zip/2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",55187


Посмотрев на распределение бакетов, можно сделать вывод о том, какие номера бакетов ```BUCKET_NUMBERS``` следует взять и какой лимит ```BUCKET_LIMIT``` на размер бакета установить. Все зависит от соотношения размеров корпусов и распределения длин предложений в них. Недостаточно механически выставлять эти гиперпараметры, иногда это может не сработать, например, когда мало данных и один из корпусов состоит преимущественно из коротких/длинных предложений.


Если из полученных бакетов не получается отобрать бакеты с хорошой точностью, то полезно вернуться на шаг назад и попробовать снизить основание логарифма, чтобы более гибко управлять длинами предложений.

В ситуации выше можно разными способами скомбинировать бакеты. Например, если оставить номера ```[2,3,4]```, то для достаточно точной балансировки придется ставить лимит на бакет около ```4000```, но можно также пожертвовать точностью на свое усмотрение и поставить размер ```4000 + K```, где ```K``` обознает перевес в сторону одного из копусов. Или можно оставить номера ```[2,3]``` и взять лимит ```45000```. 

Вообщем, здесь возможны разные варианты. Если данных много то скорее всего проблем не будет, но если данных мало и они сильно не сбалансированны изначально, то лучше перепроверить.

In [13]:
BUCKET_NUMBERS = [2, 3, 4]
BUCKET_LIMIT = 4000

In [14]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [15]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
ficbook.base.zip/2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",55187
ficbook.base.zip/3,"[10138, 10139, 10140, 10146, 10147, 10152, 101...",47695
ficbook.base.zip/4,"[10198, 21538, 21556, 21565, 93821, 93824, 938...",3896


In [ ]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/example_balanced.zip"

```BALANCED_PATH``` указывает на место, куда нужно слить корпуса. Функция ниже не является частью балансировщика и определяется там где строится бандл и выполняются другие трансфьюзы.

In [123]:
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer.select
    )

In [124]:
balancing()

2022-12-01 11:11:11.268193+00:00 INFO: Processed 7 words. 1/3531
2022-12-01 11:11:11.389087+00:00 INFO: Processed 80 words. 2/3531
2022-12-01 11:11:11.550925+00:00 INFO: Processed 178 words. 3/3531
2022-12-01 11:11:11.667973+00:00 INFO: Processed 497 words. 4/3531
2022-12-01 11:11:11.792911+00:00 INFO: Processed 599 words. 5/3531
2022-12-01 11:11:11.946499+00:00 INFO: Processed 665 words. 6/3531
2022-12-01 11:11:12.085980+00:00 INFO: Processed 863 words. 7/3531
2022-12-01 11:11:12.255750+00:00 INFO: Processed 1092 words. 8/3531
2022-12-01 11:11:12.566449+00:00 INFO: Processed 1182 words. 9/3531
2022-12-01 11:11:12.868502+00:00 INFO: Processed 1346 words. 10/3531
2022-12-01 11:11:13.130785+00:00 INFO: Processed 1547 words. 11/3531
2022-12-01 11:11:13.247006+00:00 INFO: Processed 1679 words. 12/3531
2022-12-01 11:11:13.432888+00:00 INFO: Processed 1891 words. 13/3531
2022-12-01 11:11:13.611632+00:00 INFO: Processed 2015 words. 14/3531
2022-12-01 11:11:13.770795+00:00 INFO: Processed 2330

2022-12-01 11:11:31.607754+00:00 INFO: Processed 16042 words. 119/3531
2022-12-01 11:11:31.679528+00:00 INFO: Processed 16078 words. 120/3531
2022-12-01 11:11:31.807879+00:00 INFO: Processed 16121 words. 121/3531
2022-12-01 11:11:31.904325+00:00 INFO: Processed 16121 words. 122/3531
2022-12-01 11:11:32.026494+00:00 INFO: Processed 16134 words. 123/3531
2022-12-01 11:11:32.120917+00:00 INFO: Processed 16134 words. 124/3531
2022-12-01 11:11:32.201057+00:00 INFO: Processed 16134 words. 125/3531
2022-12-01 11:11:32.285247+00:00 INFO: Processed 16143 words. 126/3531
2022-12-01 11:11:32.343367+00:00 INFO: Processed 16143 words. 127/3531
2022-12-01 11:11:32.418586+00:00 INFO: Processed 16151 words. 128/3531
2022-12-01 11:11:32.469293+00:00 INFO: Processed 16151 words. 129/3531
2022-12-01 11:11:32.531177+00:00 INFO: Processed 16151 words. 130/3531
2022-12-01 11:11:32.606232+00:00 INFO: Processed 16157 words. 131/3531
2022-12-01 11:11:32.661004+00:00 INFO: Processed 16157 words. 132/3531
2022-1

2022-12-01 11:11:41.349468+00:00 INFO: Processed 19847 words. 235/3531
2022-12-01 11:11:41.409993+00:00 INFO: Processed 19847 words. 236/3531
2022-12-01 11:11:41.465850+00:00 INFO: Processed 19847 words. 237/3531
2022-12-01 11:11:41.541134+00:00 INFO: Processed 19881 words. 238/3531
2022-12-01 11:11:41.602766+00:00 INFO: Processed 19881 words. 239/3531
2022-12-01 11:11:41.674250+00:00 INFO: Processed 19890 words. 240/3531
2022-12-01 11:11:41.728348+00:00 INFO: Processed 19890 words. 241/3531
2022-12-01 11:11:41.780612+00:00 INFO: Processed 19890 words. 242/3531
2022-12-01 11:11:41.837103+00:00 INFO: Processed 19890 words. 243/3531
2022-12-01 11:11:41.886281+00:00 INFO: Processed 19890 words. 244/3531
2022-12-01 11:11:41.942839+00:00 INFO: Processed 19890 words. 245/3531
2022-12-01 11:11:42.023592+00:00 INFO: Processed 19902 words. 246/3531
2022-12-01 11:11:42.128787+00:00 INFO: Processed 20009 words. 247/3531
2022-12-01 11:11:42.183779+00:00 INFO: Processed 20009 words. 248/3531
2022-1

2022-12-01 11:11:54.981972+00:00 INFO: Processed 25775 words. 351/3531
2022-12-01 11:11:55.146647+00:00 INFO: Processed 26015 words. 352/3531
2022-12-01 11:11:55.273318+00:00 INFO: Processed 26058 words. 353/3531
2022-12-01 11:11:55.402530+00:00 INFO: Processed 26335 words. 354/3531
2022-12-01 11:11:55.527889+00:00 INFO: Processed 26445 words. 355/3531
2022-12-01 11:11:55.616739+00:00 INFO: Processed 26483 words. 356/3531
2022-12-01 11:11:55.749451+00:00 INFO: Processed 26727 words. 357/3531
2022-12-01 11:11:55.903678+00:00 INFO: Processed 26847 words. 358/3531
2022-12-01 11:11:56.020367+00:00 INFO: Processed 26847 words. 359/3531
2022-12-01 11:11:56.324752+00:00 INFO: Processed 26986 words. 360/3531
2022-12-01 11:11:56.453989+00:00 INFO: Processed 27357 words. 361/3531
2022-12-01 11:11:56.553196+00:00 INFO: Processed 27600 words. 362/3531
2022-12-01 11:11:56.643976+00:00 INFO: Processed 27684 words. 363/3531
2022-12-01 11:11:56.713688+00:00 INFO: Processed 27705 words. 364/3531
2022-1

2022-12-01 11:12:13.757548+00:00 INFO: Processed 46320 words. 467/3531
2022-12-01 11:12:13.880921+00:00 INFO: Processed 46410 words. 468/3531
2022-12-01 11:12:14.003337+00:00 INFO: Processed 46613 words. 469/3531
2022-12-01 11:12:14.090721+00:00 INFO: Processed 46819 words. 470/3531
2022-12-01 11:12:14.212198+00:00 INFO: Processed 46854 words. 471/3531
2022-12-01 11:12:14.313116+00:00 INFO: Processed 46892 words. 472/3531
2022-12-01 11:12:14.427845+00:00 INFO: Processed 47116 words. 473/3531
2022-12-01 11:12:14.534123+00:00 INFO: Processed 47393 words. 474/3531
2022-12-01 11:12:14.623103+00:00 INFO: Processed 47553 words. 475/3531
2022-12-01 11:12:14.730991+00:00 INFO: Processed 47839 words. 476/3531
2022-12-01 11:12:14.830624+00:00 INFO: Processed 47925 words. 477/3531
2022-12-01 11:12:14.930875+00:00 INFO: Processed 48091 words. 478/3531
2022-12-01 11:12:15.030947+00:00 INFO: Processed 48317 words. 479/3531
2022-12-01 11:12:15.127121+00:00 INFO: Processed 48405 words. 480/3531
2022-1

...

2022-12-01 11:17:16.435367+00:00 INFO: Processed 414932 words. 3064/3531
2022-12-01 11:17:16.509361+00:00 INFO: Processed 415004 words. 3065/3531
2022-12-01 11:17:16.573160+00:00 INFO: Processed 415063 words. 3066/3531
2022-12-01 11:17:16.664512+00:00 INFO: Processed 415712 words. 3067/3531
2022-12-01 11:17:16.782022+00:00 INFO: Processed 416325 words. 3068/3531
2022-12-01 11:17:16.856307+00:00 INFO: Processed 416503 words. 3069/3531
2022-12-01 11:17:16.937475+00:00 INFO: Processed 416879 words. 3070/3531
2022-12-01 11:17:17.010449+00:00 INFO: Processed 417054 words. 3071/3531
2022-12-01 11:17:17.093482+00:00 INFO: Processed 417285 words. 3072/3531
2022-12-01 11:17:17.179936+00:00 INFO: Processed 417457 words. 3073/3531
2022-12-01 11:17:17.270315+00:00 INFO: Processed 417924 words. 3074/3531
2022-12-01 11:17:17.337454+00:00 INFO: Processed 417930 words. 3075/3531
2022-12-01 11:17:17.424436+00:00 INFO: Processed 418364 words. 3076/3531
2022-12-01 11:17:17.497096+00:00 INFO: Processed 41

2022-12-01 11:17:25.870239+00:00 INFO: Writing 170 frames, 50001 words, to write 49995, to keep 6
2022-12-01 11:17:26.096072+00:00 INFO: Processed 449936 words. 3177/3531
2022-12-01 11:17:26.219883+00:00 INFO: Processed 450234 words. 3178/3531
2022-12-01 11:17:26.325130+00:00 INFO: Processed 450448 words. 3179/3531
2022-12-01 11:17:26.399542+00:00 INFO: Processed 450453 words. 3180/3531
2022-12-01 11:17:26.478787+00:00 INFO: Processed 450561 words. 3181/3531
2022-12-01 11:17:26.549869+00:00 INFO: Processed 450847 words. 3182/3531
2022-12-01 11:17:26.625347+00:00 INFO: Processed 451262 words. 3183/3531
2022-12-01 11:17:26.720114+00:00 INFO: Processed 451390 words. 3184/3531
2022-12-01 11:17:26.819684+00:00 INFO: Processed 451622 words. 3185/3531
2022-12-01 11:17:26.886766+00:00 INFO: Processed 451747 words. 3186/3531
2022-12-01 11:17:26.968254+00:00 INFO: Processed 451841 words. 3187/3531
2022-12-01 11:17:27.038048+00:00 INFO: Processed 452200 words. 3188/3531
2022-12-01 11:17:27.126892

2022-12-01 11:17:36.225946+00:00 INFO: Processed 480649 words. 3288/3531
2022-12-01 11:17:36.299170+00:00 INFO: Processed 480662 words. 3289/3531
2022-12-01 11:17:36.370349+00:00 INFO: Processed 480668 words. 3290/3531
2022-12-01 11:17:36.524107+00:00 INFO: Processed 480675 words. 3291/3531
2022-12-01 11:17:36.730497+00:00 INFO: Processed 481486 words. 3292/3531
2022-12-01 11:17:36.830373+00:00 INFO: Processed 481486 words. 3293/3531
2022-12-01 11:17:36.941490+00:00 INFO: Processed 481536 words. 3294/3531
2022-12-01 11:17:37.021522+00:00 INFO: Processed 481541 words. 3295/3531
2022-12-01 11:17:37.120319+00:00 INFO: Processed 481630 words. 3296/3531
2022-12-01 11:17:37.205128+00:00 INFO: Processed 481656 words. 3297/3531
2022-12-01 11:17:37.368434+00:00 INFO: Processed 482860 words. 3298/3531
2022-12-01 11:17:37.437481+00:00 INFO: Processed 482883 words. 3299/3531
2022-12-01 11:17:37.514148+00:00 INFO: Processed 483155 words. 3300/3531
2022-12-01 11:17:37.593364+00:00 INFO: Processed 48

2022-12-01 11:17:46.944996+00:00 INFO: Processed 506023 words. 3399/3531
2022-12-01 11:17:47.043099+00:00 INFO: Processed 506041 words. 3400/3531
2022-12-01 11:17:47.109454+00:00 INFO: Processed 506239 words. 3401/3531
2022-12-01 11:17:47.217246+00:00 INFO: Processed 507363 words. 3402/3531
2022-12-01 11:17:47.310186+00:00 INFO: Processed 507929 words. 3403/3531
2022-12-01 11:17:47.389551+00:00 INFO: Processed 508285 words. 3404/3531
2022-12-01 11:17:47.453534+00:00 INFO: Processed 508519 words. 3405/3531
2022-12-01 11:17:47.496678+00:00 INFO: Processed 508519 words. 3406/3531
2022-12-01 11:17:47.571957+00:00 INFO: Processed 508705 words. 3407/3531
2022-12-01 11:17:47.652756+00:00 INFO: Processed 508786 words. 3408/3531
2022-12-01 11:17:47.753461+00:00 INFO: Processed 508812 words. 3409/3531
2022-12-01 11:17:47.822423+00:00 INFO: Processed 508862 words. 3410/3531
2022-12-01 11:17:47.895344+00:00 INFO: Processed 508944 words. 3411/3531
2022-12-01 11:17:47.970325+00:00 INFO: Processed 50

2022-12-01 11:17:57.941031+00:00 INFO: Processed 535217 words. 3512/3531
2022-12-01 11:17:58.023017+00:00 INFO: Processed 535542 words. 3513/3531
2022-12-01 11:17:58.101163+00:00 INFO: Processed 536040 words. 3514/3531
2022-12-01 11:17:58.173395+00:00 INFO: Processed 536197 words. 3515/3531
2022-12-01 11:17:58.278895+00:00 INFO: Processed 536991 words. 3516/3531
2022-12-01 11:17:58.555105+00:00 INFO: Processed 539811 words. 3517/3531
2022-12-01 11:17:58.664554+00:00 INFO: Processed 540189 words. 3518/3531
2022-12-01 11:17:58.761572+00:00 INFO: Processed 540297 words. 3519/3531
2022-12-01 11:17:58.860239+00:00 INFO: Processed 540842 words. 3520/3531
2022-12-01 11:17:58.935381+00:00 INFO: Processed 541140 words. 3521/3531
2022-12-01 11:17:59.011206+00:00 INFO: Processed 541549 words. 3522/3531
2022-12-01 11:17:59.081859+00:00 INFO: Processed 541925 words. 3523/3531
2022-12-01 11:17:59.154413+00:00 INFO: Processed 542270 words. 3524/3531
2022-12-01 11:17:59.229717+00:00 INFO: Processed 54

In [126]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

100%|███████████████████████████████████████████| 11/11 [00:01<00:00,  8.67it/s]


In [129]:
lengths

defaultdict(int, {'books.base.zip': 12000, 'ficbook.base.zip': 11896})

Теперь все хорошо, и можно делать последующую обработку.